In [140]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import OneHotEncoder
import os
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [141]:
# normalize the data ?
# noramlize the rates between 0 and 1

# don't keep the names
# player positions: remove it (remove nation position) and keep club position --> prescription: constraint on the number of players per zone. Limit to Attack / Midfield / Defend / Goalkeeper
# keep overall and potential, maybe replace potential by potential - overall
# keep club joined year, and club contract valid year
# work rate: replace by two columns, in numerical values ? or remove it
# get rid of improvements 
# keep league levels

# remove ids, urls, club / league names, club_loaned_from, nationalities / nations (id, name etc.), player tags and player traits, 

In [142]:
# define function so we can easily apply it to all the datasets
def data_preprocessing(data):
    
    # display all columns of the dataframe
    pd.set_option('display.max_columns', None)

    # remove unnecessary columns
    data.drop(['sofifa_id', 'player_url', 'long_name', 'club_position', 'dob', 'club_team_id', 
        #'club_name',
        'league_name', 'club_jersey_number', 'club_loaned_from', 'nationality_id', 'nationality_name', 
        'nation_team_id', 'nation_position', 'nation_jersey_number', 'body_type', 'real_face', 'player_tags', 'player_traits',
        'goalkeeping_speed', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm',
        'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'player_face_url', 'club_logo_url', 'club_flag_url', 
        'nation_logo_url', 'nation_flag_url'], axis = 1, inplace = True)

    # instead of club position, I keep player_positions and just pick the first element. The reason is that in club_position there are also SUB and RES
    # keep just the first position
    data['player_positions'] = data['player_positions'].apply(lambda x: x.split(',')[0])    

    # translate club position to role, dividing goalkeepers, defenders, midfielders and attackers
    attackers = ['ST', 'LW', 'RW', 'CF', 'LF', 'RF', 'RS', 'LS']
    midfielders = ['CAM', 'CM', 'CDM', 'RCM','LM', 'RM', 'LAM', 'RAM', 'LW', 'RW', 'RDM', 'LCM','LDM']
    defenders = ['CB', 'LB', 'RB', 'LCB', 'RCB', 'LWB', 'RWB']
    goalkeepers = ['GK']

    # print goalkeepers
    #print("Goalkeepers: ", data[data['player_positions'].isin(goalkeepers)].head(5))

    # create "position_A", "position_M", "position_D", "position_G" columns
    data['position_A'] = data['player_positions'].apply(lambda x: 1 if x in attackers else 0)
    data['position_M'] = data['player_positions'].apply(lambda x: 1 if x in midfielders else 0)
    data['position_D'] = data['player_positions'].apply(lambda x: 1 if x in defenders else 0)
    data['position_G'] = data['player_positions'].apply(lambda x: 1 if x in goalkeepers else 0)

    # remove player_positions
    data.drop(['player_positions'], axis = 1, inplace = True)

    # only keep year of when club was joined
    data['club_joined'] = [int(data.loc[0,'club_joined'].split('-')[0]) for i in range(len(data['club_joined']))]
    
    # one hot encoding of preferred foot, 0 = left, 1 = right
    data['preferred_foot'] = data['preferred_foot'].apply(lambda x: 0 if x == 'Left' else 1)

    # ordinal encoding of work rate (1 = Low, 2 = Medium, 3 = High), split between attack and defense. 
    data['attack_work_rate'] = data['work_rate'].apply(lambda x: 1 if x.split('/')[0] == 'Low' else (2 if x.split('/')[0] == 'Medium' else 3))
    data['defense_work_rate'] = data['work_rate'].apply(lambda x: 1 if x.split('/')[1] == 'Low' else (2 if x.split('/')[1] == 'Medium' else 3))
    data.drop(['work_rate'], axis = 1, inplace = True)


    # drop rows with nan values if there are not in the authorized columns
    authorized_nan_columns = ["passing", "physic", "defending", "dribbling", "shooting", "pace"]
    data.dropna(subset = [col for col in data.columns if col not in authorized_nan_columns], inplace = True)

    # return the dataframe
    return data

In [143]:
# save dataframes from different years. Each entry of the dataframe has the cleaned
# fifa data for a specific year
total_data = []

for year in range(20, 22+1):
    # read the data
    path = '../data/fifa/players_{}.csv'.format(year)
    data = pd.read_csv(path)

    # preprocess the data
    total_data.append(data_preprocessing(data))

# for each year, add the year column to the dataframe
for i in range(len(total_data)):
    total_data[i]['year'] = 2000 + i

/var/folders/lf/rts57f595bn_1gbpcfsjp4br0000gn/T/ipykernel_69031/1986099377.py:8: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


####################################################################################################

Now, cleaning of the transfer data

In [135]:
# read all transfer files
path = "../data/Transfers/"
csv_files = glob.glob(os.path.join(path, "*.csv"))

In [136]:
# create unique dataframe with all the transfers
transfers_list = []

# loop over the list of csv files
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f)

    # append the dataframe to the list
    transfers_list.append(df)

transfers = pd.concat(transfers_list).reset_index(drop = True)

In [137]:
# only keep transfers happened in 2020, 2021, 2022
transfers = transfers.loc[transfers['year'].isin([2020, 2021, 2022])]

# reset index
transfers.reset_index(inplace = True, drop = True)

# just keep transfer movement in, because there should be a match between in and out, but the ones that just have out mean they are not playing
transfers = transfers[transfers["transfer_movement"] == "in"]

# remove columns we are not interested in
transfers.drop(['age', 'position', 
    #'club_involved_name', 
    'transfer_movement', 'league_name', 'season'], axis = 1, inplace = True)

# one hot encoding of transfer period, 1 = summer, 0 = winter
transfers['transfer_period'] = transfers['transfer_period'].apply(lambda x: 1 if x.lower() == 'summer' else 0)

# check if loan is in fee
transfers['loan'] = transfers['fee'].apply(lambda x: 1 if 'loan' in x.lower() else 0)

# get rid of fee column, and rename fee_cleaned to fee
transfers.drop(['fee'], axis = 1, inplace = True)
transfers.rename(columns = {'fee_cleaned': 'fee'}, inplace = True)

# get rid of loan transfers
transfers = transfers[transfers['loan'] == 0]
transfers.drop(['loan'], axis = 1, inplace = True)

# rename club_name to new_club and club_involved_name to old_club
transfers.rename(columns = {'club_name': 'new_club'}, inplace = True)
transfers.rename(columns = {'club_involved_name': 'old_club'}, inplace = True)

# drop duplicates, meaning player_name that appears twice. We keep the last one since it's the most recent transfer
#transfers.drop_duplicates(subset = ['player_name'], keep = 'last', inplace = True)

# reset index
transfers.reset_index(inplace = True, drop = True)

# remove NaN values
transfers.dropna(inplace = True)

# Now we need to join the two datasets based on player, if there is a doubt, we keep the club_name, if still we are not sure, we drop it

## Analysis

In [138]:
transfers_22 = transfers[transfers['year'] == 2022].reset_index(drop = True)
transfers_21 = transfers[transfers['year'] == 2021].reset_index(drop = True)
transfers_20 = transfers[transfers['year'] == 2020].reset_index(drop = True)

# create ratings_20, ratings_21, ratings_22
ratings_22 = total_data[2].reset_index(drop = True)
ratings_20 = total_data[0].reset_index(drop = True)
ratings_21 = total_data[1].reset_index(drop = True)

# add 20 years in the year column for each ratings
ratings_20['year'] = ratings_20['year'] + 20
ratings_21['year'] = ratings_21['year'] + 20
ratings_22['year'] = ratings_22['year'] + 20

In [139]:
# print ratings files to csv
ratings_22.to_csv('../data/Final/ratings/ratings_22.csv', index = False)
ratings_21.to_csv('../data/Final/ratings/ratings_21.csv', index = False)
ratings_20.to_csv('../data/Final/ratings/ratings_20.csv', index = False)

In [13]:
#!pip install fuzzywuzzy

In [14]:
# player_name from transfers is not the same as player_name from fifa data, so we need to match them
# we can use a metric to measure the similarity between the two names, and then match the ones that are the most similar
# use fuzzywuzzy to get the similarity between two strings
from fuzzywuzzy import fuzz

# function to get the similarity between two strings
def get_similarity(a, b):
    return fuzz.token_sort_ratio(a, b)
    

# function to get the most similar string from a list of strings
def get_most_similar(string, string_list):
    # get the similarity between the string and each string in the list
    similarity = [get_similarity(string, s) for s in string_list]

    # get the index of the most similar string
    max_index = np.argmax(similarity)

    # return the most similar string
    return string_list[max_index]


def get_short_name(player_name):
    # if there is no firts name, return the last name
    if len(player_name.split()) == 1:
        return player_name
    # else, the first name is the first word, and the last name is all the other words
    else:
        first_name = player_name.split()[0]
        last_name = player_name.split()[1:]
        return first_name[0] + ". " + " ".join(last_name)


In [15]:
def map_transfers(transfers, ratings):
    # let's try to map the club names from the transfers dataframe to the club names from the fifa dataframe
    clubs_names = transfers.old_club.unique()
    fifa_clubs = ratings.club_name.unique()

    # create a dictionary to map the club names
    club_name_dict = {}

    # loop over the list of clubs
    for club in tqdm(clubs_names):
        # get the most similar club name from the fifa dataframe
        most_similar_club = get_most_similar(club, fifa_clubs)

        # add the mapping to the dictionary
        club_name_dict[club] = most_similar_club

    # map the club names
    transfers['club_name'] = transfers['old_club'].map(club_name_dict)



    # let's try to map the player names from the transfers dataframe to the player names from the fifa dataframe
    # First possibility to map them: in the ratings dataframe, the short_name looks like "L. Messi", so we can use the first letter of the first name and the last name to match the players
    # if not, we can use the previous functions to match the players

    # get the short name of the players
    transfers['short_name'] = transfers['player_name'].apply(get_short_name)
    
    return transfers


# second function, only if necessary
def add_name_mapping(transfers, ratings):
     # let's try to map the player names from the transfers dataframe to the player names from the fifa dataframe
    # use the get_most_similar function to get the most similar player name from the fifa dataframe
    # create an empty column "player_name_mapped" that we will add to transfers at the end
    player_name_mapped = []

    transfers_info = transfers[['player_name', 'club_name']].values
    ratings_info = ratings[['short_name', 'club_name']].values

    # loop over the list of players
    for player in tqdm(transfers_info):
        # check only the players that play in the same club from ratings
        ratings_players = ratings_info[ratings_info[:, 1] == player[1]]

        # get the most similar player name from the fifa dataframe
        most_similar_player = get_most_similar(player[0], ratings_players[:, 0])

        # add the most similar player name to the list
        player_name_mapped.append(most_similar_player)

    # add the list to the transfers dataframe
    transfers['player_name_mapped'] = player_name_mapped

    return transfers

In [16]:
# let's map the transfers to the fifa data
transfers_22 = map_transfers(transfers_22, ratings_22)
transfers_21 = map_transfers(transfers_21, ratings_21)
transfers_20 = map_transfers(transfers_20, ratings_20)


100%|██████████| 498/498 [00:02<00:00, 225.39it/s]


In [17]:
# function to merge the transfers with the fifa data
def merge(transfers, ratings):
    # keep in memory the initial transfers columns
    transfers_columns = transfers.columns

    # first, try to merge using the short_name and club_name
    merged = pd.merge(transfers, ratings, on = ['short_name', 'club_name'], how = 'left', indicator=True)

    # create a dataframe for the rows where merge==both and one for the rows where merge==left_only
    first_merge_success = merged[merged['_merge'] == 'both'].reset_index(drop = True)
    first_merge_fail = merged[merged['_merge'] == 'left_only'].reset_index(drop = True)

    # drop the _merge column
    first_merge_success.drop(['_merge'], axis = 1, inplace = True)
    first_merge_fail.drop(['_merge'], axis = 1, inplace = True)
    
    # get the transfers data corresponding to the first merge fail
    first_merge_fail_transfers = first_merge_fail[transfers_columns].reset_index(drop = True)

    # use the add_name_mapping function to map the player names in first_merge_fail_transfers
    first_merge_fail_transfers = add_name_mapping(first_merge_fail_transfers, ratings)
    # rename the short name to previous_short_name and the player_name_mapped to short_name
    first_merge_fail_transfers.rename(columns = {'short_name': 'previous_short_name', 'player_name_mapped': 'short_name'}, inplace = True)

    # try a new merge, and only keep the inner merge
    second_merge = pd.merge(first_merge_fail_transfers, ratings, on = ['short_name', 'club_name'], how = 'inner')

    # similarities
    first_merge_success['name_similarity'] = first_merge_success.apply(lambda x: get_similarity(x['player_name'], x['short_name']), axis = 1)
    first_merge_success['club_similarity'] = first_merge_success.apply(lambda x: get_similarity(x['old_club'], x['club_name']), axis = 1)
    second_merge['name_similarity'] = second_merge.apply(lambda x: get_similarity(x['player_name'], x['short_name']), axis = 1)
    second_merge['club_similarity'] = second_merge.apply(lambda x: get_similarity(x['old_club'], x['club_name']), axis = 1)
    
    # drop the columns for which the similarity score between old_club and club_name is less than 75% of the similarity score in the first merge success
    club_similarity_threshold = first_merge_success['club_similarity'].quantile(0.25)
    second_merge = second_merge[second_merge['club_similarity'] >= club_similarity_threshold].reset_index(drop = True)
    
    # drop the columns for which the similarity score between player_name and short_name is less than 75% of the similarity score in the first merge success
    name_similarity_threshold = first_merge_success['name_similarity'].quantile(0.25)
    second_merge = second_merge[second_merge['name_similarity'] >= name_similarity_threshold].reset_index(drop = True)

    # concatenate the two dataframes
    final_merge = pd.concat([first_merge_success, second_merge], axis = 0).reset_index(drop = True)
    
    # put previous_short_name and similarity in the first columns
    final_merge = final_merge[['previous_short_name', 'name_similarity', 'club_similarity'] + final_merge.columns[:-3].tolist()]

    # drop duplicates
    final_merge.drop_duplicates(inplace = True)

    return final_merge

In [18]:
merged_22 = merge(transfers_22, ratings_22)
merged_21 = merge(transfers_21, ratings_21)
merged_20 = merge(transfers_20, ratings_20)

# order by similarity, to check if the merge was successful
merged_22.sort_values(by = 'name_similarity', ascending = True, inplace = True)
merged_21.sort_values(by = 'name_similarity', ascending = True, inplace = True)
merged_20.sort_values(by = 'name_similarity', ascending = True, inplace = True)

# print merged_22 where previous_short_name is not null
#merged_22[merged_22['previous_short_name'].notnull()].head(40)
#merged_21[merged_21['previous_short_name'].notnull()].head(40)
#merged_20[merged_20['previous_short_name'].notnull()].head(40)

100%|██████████| 1270/1270 [00:00<00:00, 2593.97it/s]


In [19]:
# print all shapes
print(merged_22.shape)
print(merged_21.shape)
print(merged_20.shape)

(459, 72)
(480, 72)
(405, 72)


In [32]:
merged_22[(merged_22.fee==0) & (merged_22.club_contract_valid_until==2022)]
#merged_21[merged_21.fee==0].club_contract_valid_until.value_counts()
#merged_20[merged_20.fee==0].club_contract_valid_until.value_counts()

merged_22[merged_22.fee==0].shape[0] / merged_22.shape[0] * 100
merged_22.head()

,previous_short_name,name_similarity,club_similarity,new_club,player_name,old_club,transfer_period,fee,year,club_name,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_joined,club_contract_valid_until,preferred_foot,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,position_A,position_D,position_G,position_M,attack_work_rate,defense_work_rate
107,NaN,50,100,Sunderland AFC,Abdoullah Ba,AC Le Havre,1,1.0,2022,Le Havre AC,A. Ba,63.0,76.0,1100000.0,500.0,17.0,180.0,75.0,2.0,2021.0,2023.0,1.0,3.0,3.0,1.0,2600000.0,66.0,56.0,61.0,66.0,53.0,62.0,54.0,54.0,47.0,66.0,36.0,66.0,60.0,42.0,59.0,67.0,68.0,65.0,70.0,61.0,63.0,63.0,67.0,69.0,60.0,59.0,58.0,56.0,59.0,65.0,48.0,64.0,49.0,58.0,52.0,5.0,14.0,8.0,7.0,7.0,0.0,0.0,0.0,1.0,2.0,2.0
603,NaN,55,67,GD Estoril Praia,Titouan Thomas,Olymp. Lyon B,1,0.0,2022,Olympique Lyonnais,T. Thomas,59.0,76.0,575000.0,5000.0,19.0,183.0,74.0,1.0,2021.0,2022.0,1.0,3.0,3.0,1.0,1500000.0,55.0,51.0,57.0,59.0,56.0,64.0,41.0,48.0,56.0,66.0,38.0,58.0,48.0,41.0,63.0,62.0,55.0,55.0,52.0,58.0,60.0,61.0,61.0,57.0,68.0,52.0,65.0,58.0,48.0,59.0,50.0,58.0,58.0,54.0,51.0,10.0,8.0,7.0,14.0,8.0,0.0,0.0,0.0,1.0,2.0,2.0
598,NaN,56,71,SL Benfica,Alexander Bah,Slavia Prague,1,8.0,2022,SK Slavia Praha,A. Bah,75.0,81.0,8500000.0,800.0,23.0,183.0,76.0,1.0,2021.0,2025.0,1.0,3.0,3.0,1.0,19600000.0,92.0,60.0,66.0,70.0,69.0,78.0,69.0,55.0,54.0,65.0,54.0,70.0,59.0,61.0,61.0,67.0,90.0,93.0,77.0,71.0,70.0,78.0,70.0,85.0,74.0,54.0,83.0,73.0,68.0,69.0,51.0,68.0,67.0,73.0,70.0,6.0,12.0,12.0,9.0,13.0,0.0,1.0,0.0,0.0,3.0,3.0
499,NaN,57,90,FC Emmen,Metehan Güçlü,Stade Rennais,1,0.0,2022,Stade Rennais FC,M. Güçlü,61.0,66.0,500000.0,7000.0,22.0,182.0,70.0,1.0,2021.0,2023.0,0.0,2.0,2.0,1.0,988000.0,75.0,60.0,54.0,61.0,23.0,56.0,42.0,60.0,62.0,60.0,56.0,59.0,47.0,48.0,49.0,60.0,73.0,76.0,75.0,58.0,66.0,60.0,71.0,48.0,64.0,59.0,44.0,21.0,59.0,60.0,66.0,51.0,18.0,16.0,20.0,7.0,13.0,7.0,10.0,13.0,1.0,0.0,0.0,0.0,2.0,2.0
464,NaN,60,100,Newcastle United,Alexander Isak,Real Sociedad,1,70.0,2022,Real Sociedad,A. Isak,82.0,86.0,45000000.0,37000.0,21.0,192.0,77.0,1.0,2021.0,2026.0,1.0,5.0,4.0,2.0,97900000.0,84.0,82.0,68.0,79.0,29.0,69.0,61.0,83.0,77.0,79.0,83.0,79.0,70.0,57.0,47.0,82.0,82.0,85.0,79.0,81.0,66.0,84.0,71.0,70.0,75.0,77.0,52.0,29.0,86.0,76.0,75.0,77.0,25.0,21.0,18.0,7.0,6.0,9.0,7.0,11.0,1.0,0.0,0.0,0.0,3.0,1.0


In [22]:
# print files to csv
merged_22.to_csv('../data/Final/merged/merged_22.csv', index = False)
merged_21.to_csv('../data/Final/merged/merged_21.csv', index = False)
merged_20.to_csv('../data/Final/merged/merged_20.csv', index = False)

In [45]:
# compare value_eur and release_clause_eur in merged_22
merged_22[['value_eur', 'release_clause_eur']].head(20)
# compute the difference between value_eur and release_clause_eur
diff = sum((merged_22['value_eur'] - merged_22['release_clause_eur']) > 0)
diff

0